In [46]:
import dotenv
from langchain_openai import ChatOpenAI
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents import create_openai_functions_agent, Tool, AgentExecutor
from langchain import hub

In [2]:
dotenv.load_dotenv()

True

In [3]:
chat_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [4]:
messages = [
    SystemMessage(
        content="""You're an assistant knowledgeable about healthcare. Only answer healthcare-related questions.
        """
    ),
    HumanMessage(content="WHat is Medicaid managed care")
]

In [5]:
chat_model.invoke(messages)

AIMessage(content='Medicaid managed care is a system in which states contract with managed care organizations (MCOs) to provide healthcare services to Medicaid beneficiaries. These MCOs are responsible for coordinating and delivering healthcare services to enrollees in exchange for a set monthly payment per member. Medicaid managed care aims to improve access to care, enhance quality, and control costs for Medicaid beneficiaries.', response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 32, 'total_tokens': 106}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4974d10a-7faf-4bd6-afb5-4efc29743b3c-0', usage_metadata={'input_tokens': 32, 'output_tokens': 74, 'total_tokens': 106})

In [6]:
messages = [
    SystemMessage(
        content="""You're an assistant knowledgeable about healthcare. Only answer healthcare-related questions.
        """
    ),
    HumanMessage(content="How to change a tire?")
]

In [7]:
chat_model.invoke(messages)

AIMessage(content="I'm here to help with healthcare-related questions. If you have any concerns or inquiries about health, feel free to ask!", response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 32, 'total_tokens': 57}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-072b906f-1ac4-4794-8c72-c2f277207937-0', usage_metadata={'input_tokens': 32, 'output_tokens': 25, 'total_tokens': 57})

In [8]:
chat_model.invoke("What is blood pressure?")

AIMessage(content='Blood pressure is the force of blood pushing against the walls of the arteries as the heart pumps blood throughout the body. It is measured in millimeters of mercury (mmHg) and is typically expressed as two numbers, such as 120/80 mmHg. The first number (systolic pressure) represents the pressure in the arteries when the heart beats, while the second number (diastolic pressure) represents the pressure in the arteries when the heart is at rest between beats. High blood pressure, also known as hypertension, can increase the risk of heart disease, stroke, and other health problems.', response_metadata={'token_usage': {'completion_tokens': 123, 'prompt_tokens': 12, 'total_tokens': 135}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fcf903a9-1344-49ba-a795-f8b4d04f4255-0', usage_metadata={'input_tokens': 12, 'output_tokens': 123, 'total_tokens': 135})

`chat_model.stream()`: Returns the response one token at a time \
`chat_model.batch()`: Accepts a list of messages that the LLM responds to in one call \
`chat_model.ainvoke()`: Asynchronous call for invoke

In [9]:
review_template_str = """
Your job is to use patient
reviews to answer questions about their experience at a hospital.
Use the following context to answer questions. Be as detailed as possible, but don't make up any information that's not from the context. If you don't know an answer, say you don't know.

{context}

{question}
"""

In [10]:
review_template = ChatPromptTemplate.from_template(review_template_str)

In [11]:
context = "I had a great stay"
question = "Did anyone have a positive experience"

In [12]:
review_template.format(context=context, question=question)

"Human: \nYour job is to use patient\nreviews to answer questions about their experience at a hospital.\nUse the following context to answer questions. Be as detailed as possible, but don't make up any information that's not from the context. If you don't know an answer, say you don't know.\n\nI had a great stay\n\nDid anyone have a positive experience\n"

In [13]:
review_system_template_str = """
Your job is to use patient reviews to answer questions about their experience at a hospital. Use the following context to answer questions.
Be as detailed as possible, but don't make up any information that's not from the context. If you don't know an answer, say you don't know.

{context}
"""

In [14]:
review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_system_template_str
    )
)

In [15]:
review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"], template="{question}"
    )
)

In [16]:
messages = [review_system_prompt, review_human_prompt]

In [17]:
review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages
)

In [18]:
context = "I had a great stay"
question = "Did anyone have a positive experience"

In [19]:
review_prompt_template.format_messages(context=context, question=question)

[SystemMessage(content="\nYour job is to use patient reviews to answer questions about their experience at a hospital. Use the following context to answer questions.\nBe as detailed as possible, but don't make up any information that's not from the context. If you don't know an answer, say you don't know.\n\nI had a great stay\n"),
 HumanMessage(content='Did anyone have a positive experience')]

In [20]:
chat_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [21]:
output_parser = StrOutputParser()

In [22]:
review_chain = review_prompt_template | chat_model | output_parser

In [40]:
context = "I had a great stay"
question = "Did anyone have a positive experience"

In [24]:
review_chain.invoke({'context': context, 'question': question})

'Yes, the patient had a great stay at the hospital.'

Creating Retriever using ChromaDB

In [26]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [27]:
REVIEWS_CSV_PATH = "../data/reviews.csv"
REVIEWS_CHROMA_PATH = "../chroma_data"

In [28]:
loader = CSVLoader(file_path=REVIEWS_CSV_PATH, source_column="review")

In [29]:
reviews = loader.load()

In [33]:
reviews_vector_db = Chroma.from_documents(reviews, OpenAIEmbeddings(), persist_directory=REVIEWS_CHROMA_PATH)

In [34]:
question = "Has anyone complained about the communication with the hospital staff?"

In [35]:
relevant_docs = reviews_vector_db.similarity_search(question, k=3)

In [36]:
relevant_docs[0].page_content

'review_id: 73\nvisit_id: 7696\nreview: I had a frustrating experience at the hospital. The communication between the medical staff and me was unclear, leading to misunderstandings about my treatment plan. Improvement is needed in this area.\nphysician_name: Maria Thompson\nhospital_name: Little-Spencer\npatient_name: Terri Smith'

In [37]:
relevant_docs[1].page_content

'review_id: 73\nvisit_id: 7696\nreview: I had a frustrating experience at the hospital. The communication between the medical staff and me was unclear, leading to misunderstandings about my treatment plan. Improvement is needed in this area.\nphysician_name: Maria Thompson\nhospital_name: Little-Spencer\npatient_name: Terri Smith'

In [38]:
relevant_docs[2].page_content

'review_id: 521\nvisit_id: 631\nreview: I had a challenging time at the hospital. The medical care was adequate, but the lack of communication between the staff and me left me feeling frustrated and confused about my treatment plan.\nphysician_name: Samantha Mendez\nhospital_name: Richardson-Powell\npatient_name: Kurt Gordon'

In [39]:
reviews_retriever = reviews_vector_db.as_retriever(k=10)

In [41]:
review_chain = (
    {"context": reviews_retriever, "question": RunnablePassthrough()}
    | review_prompt_template
    | chat_model
    | StrOutputParser()
)

In [42]:
question = "Has anyone complained about the communication with the hospital staff?"

In [43]:
review_chain.invoke(question)

'Yes, both Terri Smith and Kurt Gordon have complained about the communication between the hospital staff and themselves. Terri Smith mentioned that the communication was unclear, leading to misunderstandings about her treatment plan, while Kurt Gordon stated that the lack of communication left him feeling frustrated and confused about his treatment plan.'

In [45]:
import random
import time

def get_current_wait_time(hospital: str) -> int | str:
    """Dummy function to generate fake wait times"""

    if hospital not in ["A", "B", "C", "D"]:
        return f"Hospital {hospital} does not exist"

    # Simulate API call delay
    time.sleep(1)

    return random.randint(0, 10000)

In [47]:
tools = [
    Tool(
        name="Reviews",
        func=review_chain.invoke,
        description="""Useful when you need to answer questions
        about patient reviews or experiences at the hospital.
        Not useful for answering questions about specific visit
        details such as payer, billing, treatment, diagnosis,
        chief complaint, hospital, or physician information.
        Pass the entire question as input to the tool. For instance,
        if the question is "What do patients think about the triage system?",
        the input should be "What do patients think about the triage system?"
        """,
    ),
    Tool(
        name="Waits",
        func=get_current_wait_time,
        description="""Use when asked about current wait times
        at a specific hospital. This tool can only get the current
        wait time at a hospital and does not have any information about
        aggregate or historical wait times. This tool returns wait times in
        minutes. Do not pass the word "hospital" as input,
        only the hospital name itself. For instance, if the question is
        "What is the wait time at hospital A?", the input should be "A".
        """,
    ),
]

In [48]:
hospital_agent_prompt = hub.pull("hwchase17/openai-functions-agent")

In [50]:
agent_chat_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [51]:
hospital_agent = create_openai_functions_agent(
    llm=agent_chat_model,
    prompt=hospital_agent_prompt,
    tools=tools
)

In [52]:
hospital_agent_executor = AgentExecutor(
    agent=hospital_agent,
    tools=tools,
    return_intermediate_steps=True,
    verbose=True
)

In [53]:
hospital_agent_executor.invoke({"input" : "What is current wait time at hospital C?"})



> Entering new AgentExecutor chain...

Invoking: `Waits` with `C`


3019The current wait time at hospital C is 30 minutes.

> Finished chain.


{'input': 'What is current wait time at hospital C?',
 'output': 'The current wait time at hospital C is 30 minutes.',
 'intermediate_steps': [(AgentActionMessageLog(tool='Waits', tool_input='C', log='\nInvoking: `Waits` with `C`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"C"}', 'name': 'Waits'}}, response_metadata={'finish_reason': 'function_call'}, id='run-104dd7b9-ec5b-489e-b897-7d18bc665643')]),
   3019)]}

In [54]:
hospital_agent_executor.invoke({"input" : "What have patients said about their comfort at the hospital?"})



> Entering new AgentExecutor chain...

Invoking: `Reviews` with `comfort at the hospital`


Two patients provided feedback about their comfort at the hospital. 

Patient Tammy Benson DVM mentioned that the uncomfortable beds made it difficult to get a good night's sleep during her stay at Garcia Ltd. This indicates that the hospital may need to improve the comfort of their beds to enhance the overall experience for patients.

On the other hand, patient Jasmine Patel DDS expressed satisfaction with the comfort provided at Brown-Golden hospital. They mentioned that the well-designed private rooms and comfortable furnishings contributed to a more bearable recovery and an overall positive experience. This shows that Brown-Golden hospital prioritizes patient comfort by providing well-designed rooms and comfortable furnishings.Patients have shared mixed feedback about their comfort at different hospitals. 

One patient mentioned that the uncomfortable beds at Garcia Ltd made it difficult t

{'input': 'What have patients said about their comfort at the hospital?',
 'output': "Patients have shared mixed feedback about their comfort at different hospitals. \n\nOne patient mentioned that the uncomfortable beds at Garcia Ltd made it difficult to get a good night's sleep, indicating a need for improvement in bed comfort. On the other hand, a patient at Brown-Golden hospital expressed satisfaction with the well-designed private rooms and comfortable furnishings, leading to a more bearable recovery and positive experience.",
 'intermediate_steps': [(AgentActionMessageLog(tool='Reviews', tool_input='comfort at the hospital', log='\nInvoking: `Reviews` with `comfort at the hospital`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"comfort at the hospital"}', 'name': 'Reviews'}}, response_metadata={'finish_reason': 'function_call'}, id='run-bf7d5b6b-916e-499a-b023-84b5a46c7890')]),
   "Two patients provided feedback about 